# Imports

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import importlib
import data_loading
import data_processing

importlib.reload(data_loading)
importlib.reload(data_processing)

from data_loading import load_dataset, create_input_space, augment_data
from data_processing import preprocess_signals, normalize_data

In [3]:
data = load_dataset(signal_names=['ecg', 'gsr'])

In [4]:
print(len(data))
print(len(data[0]['signals']['ecg']))

8600
2816


# Preprocessing

In [5]:
data_filtered = preprocess_signals(data, 512, 256)

In [6]:
X, y = create_input_space(data_filtered)

(8600, 1408, 2)
(8600,)


In [7]:
X = normalize_data(X, local=False)

In [8]:
augmented_X, augmented_y = augment_data(X, y)

(77400, 1152, 2)
(77400,)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(augmented_X, augmented_y, test_size=0.05, random_state=42)

# Model

In [36]:
from tensorflow.keras.layers import Input, Dense, Reshape, Conv1D, MaxPooling1D, UpSampling1D, Multiply, GaussianNoise, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def squeeze_excite_block(input, ratio=16):
    """Create a squeeze and excitation block."""
    filters = input.shape[-1]
    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    return Multiply()([input, se])

def create_encoder(input_shape, noise_level=0.1):
    """Create a more compressed denoising encoder with squeeze-excite blocks."""
    inputs = Input(shape=input_shape)
    x = GaussianNoise(noise_level)(inputs)

    # First convolutional block with squeeze-excite
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)
    x = MaxPooling1D(2, padding='same')(x)  # Reducing dimensionality

    # Second convolutional block with squeeze-excite
    x = Conv1D(16, 3, activation='relu', padding='same', strides=2)(x)
    x = squeeze_excite_block(x)
    x = MaxPooling1D(2, padding='same')(x)  # Further reducing dimensionality

    # Third convolutional block for more compression
    x = Conv1D(8, 3, activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)
    encoded = MaxPooling1D(2, padding='same')(x)  # Final reduction

    return Model(inputs, encoded, name='encoder')


def create_decoder(encoded_shape):
    """Create a decoder to match the updated, more compressed encoder with squeeze-excite blocks."""
    encoded_input = Input(shape=encoded_shape)

    # First upsample block with squeeze-excite
    x = Conv1D(8, 3, activation='relu', padding='same')(encoded_input)
    x = squeeze_excite_block(x)
    x = UpSampling1D(2)(x)  # Upsampling to 144

    # Second upsample block with squeeze-excite
    x = Conv1D(16, 3, activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)
    x = UpSampling1D(2)(x)  # Upsampling to 288

    # Third upsample block with squeeze-excite
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)
    x = UpSampling1D(4)(x)  # Upsampling to 1152

    # Final reconstruction layer
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)

    return Model(encoded_input, decoded, name='decoder')


In [37]:
# Signal specific input shapes
signal_shapes = {
    'ECG': (1152, 1),
    'GSR': (1152, 1),
    # Add more signals as needed
}

# Creating a dictionary to hold each signal's autoencoder
autoencoders = {}

for signal_type, input_shape in signal_shapes.items():
    encoder = create_encoder(input_shape)
    decoder = create_decoder(encoder.output_shape[1:])

    autoencoder_input = Input(shape=input_shape)
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)

    autoencoder = Model(autoencoder_input, decoded, name=f'autoencoder_{signal_type}')
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    autoencoders[signal_type] = autoencoder
    autoencoder.summary()

Model: "autoencoder_ECG"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_41 (InputLayer)       [(None, 1152, 1)]         0         
                                                                 
 encoder (Functional)        (None, 72, 8)             2291      
                                                                 
 decoder (Functional)        (None, 1152, 1)           2484      
                                                                 
Total params: 4775 (18.65 KB)
Trainable params: 4775 (18.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "autoencoder_GSR"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_44 (InputLayer)       [(None, 1152, 1)]         0         
                                                          

In [31]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Assuming X_train and y_train are dictionaries with keys corresponding to signal types
# and values being the training data for each signal
X_train = {
    'ECG': X_train[:,:,0],  
    'GSR': X_train[:,:,1]
    # Add more signals as needed
}

# In case of autoencoders, usually, the target is the same as the input (for reconstruction tasks)
y_train = X_train

In [38]:
# Define the number of epochs and batch size for training
epochs = 50
batch_size = 32

# Training each autoencoder
for signal_type, autoencoder in autoencoders.items():
    print(f"Training autoencoder for {signal_type}...")
    autoencoder.fit(
        X_train[signal_type], y_train[signal_type],
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,  # Assuming you want to use 20% of the data for validation
        shuffle=True
    )

Training autoencoder for ECG...
Epoch 1/50
1839/1839 [==============================] - 152s 78ms/step - loss: 0.0048 - val_loss: 2.0635e-04
Epoch 2/50
1839/1839 [==============================] - 144s 78ms/step - loss: 3.2253e-04 - val_loss: 1.7137e-04
Epoch 3/50
1839/1839 [==============================] - 139s 76ms/step - loss: 2.7852e-04 - val_loss: 1.2343e-04
Epoch 4/50
1839/1839 [==============================] - 137s 74ms/step - loss: 2.6147e-04 - val_loss: 1.2492e-04
Epoch 5/50
1839/1839 [==============================] - 135s 73ms/step - loss: 2.5062e-04 - val_loss: 1.3623e-04
Epoch 6/50
1839/1839 [==============================] - 134s 73ms/step - loss: 2.4288e-04 - val_loss: 1.1013e-04
Epoch 7/50
1839/1839 [==============================] - 137s 74ms/step - loss: 2.3829e-04 - val_loss: 1.1507e-04
Epoch 8/50
1839/1839 [==============================] - 131s 71ms/step - loss: 2.3375e-04 - val_loss: 9.3609e-05
Epoch 9/50
1839/1839 [==============================] - 118s 64ms/st

KeyboardInterrupt: 

In [ ]:
#todo create leave one out CV for data loading - no data augmentation on test set: trim to first 4.5s
#todo create assessment function to see models performance